# (Py)Spark Exercizes
@stravanni



In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext(conf=conf)

----
# Exercises
### Try to solve the following exercizes employing the API presentet above

## A. Wordcount
1. Read the file "example.txt", containing "THE DIVINE COMEDY"
2. Select the 10 most frequent words

In [7]:
FILE_PATH = "data/"

In [9]:
# Write your solution here
# Most common words in "THE DIVINE COMEDY"
rdd = sc.textFile(FILE_PATH + "DivineComedy.txt")
res = (rdd.flatMap(lambda x: x.split())
       .map(lambda x: x.lower())
       .map(lambda x: (x, 1))
       .reduceByKey(lambda x, y: x + y)
       .sortBy(lambda x: x[1], ascending=False)
      ).take(10)
res

[('e', 3987),
 ('che', 3545),
 ('la', 2337),
 ('di', 1875),
 ('a', 1806),
 ('non', 1419),
 ('per', 1363),
 ('in', 1093),
 ('si', 1042),
 ('s�', 1038)]

## B. Estimating Pi
This code estimates π by "throwing darts" at a circle.

1. We pick random points in the unit square ((0, 0) to (1,1)) and see how many fall in the unit circle.
2. The fraction should be π / 4, so we use this to get our estimate.

In [10]:
# SOLUTION
from random import random
NUM_SAMPLES = 100000000

def sample(p):
    x, y = random(), random()
    return 1 if x*x + y*y < 1 else 0


    

sample_nums = sc.parallelize(range(0, NUM_SAMPLES))

sample = sample_nums.map(sample)
def summ(x, y):
    return x + y

count = sample.reduce(summ)
#count = sample.reduce(lambda x, y: x + y)

print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

Pi is roughly 3.142022


## C. TMax

In [11]:
import re
import sys

In [12]:
#function to extract the data from the line
#based on position and filter out the invalid records
def extractData(line):
    val = line.strip()
    (year, temp, q) = (str(val[15:19]), str(val[87:92]), str(val[92:93]))
    if (temp != "+9999" and re.match("[01459]", q)):
        return [(year, temp)]
    else:
        return []

In [17]:
#Create an RDD from the input data
weatherData = sc.textFile(FILE_PATH + "1902.txt")

#Transform the data to extract/filter and then find the max temperature
temperature_per_year = weatherData.flatMap(extractData)

In [18]:
temperature_per_year.collect()

[('1902', '-0094'),
 ('1902', '-0100'),
 ('1902', '-0117'),
 ('1902', '-0161'),
 ('1902', '-0172'),
 ('1902', '-0178'),
 ('1902', '-0178'),
 ('1902', '-0172'),
 ('1902', '-0150'),
 ('1902', '-0106'),
 ('1902', '-0094'),
 ('1902', '-0100'),
 ('1902', '-0072'),
 ('1902', '-0061'),
 ('1902', '-0094'),
 ('1902', '-0117'),
 ('1902', '-0128'),
 ('1902', '-0128'),
 ('1902', '-0144'),
 ('1902', '-0044'),
 ('1902', '-0011'),
 ('1902', '-0033'),
 ('1902', '-0017'),
 ('1902', '-0017'),
 ('1902', '-0061'),
 ('1902', '-0072'),
 ('1902', '-0089'),
 ('1902', '-0078'),
 ('1902', '-0067'),
 ('1902', '-0067'),
 ('1902', '-0089'),
 ('1902', '-0117'),
 ('1902', '-0122'),
 ('1902', '-0100'),
 ('1902', '-0117'),
 ('1902', '-0117'),
 ('1902', '-0078'),
 ('1902', '-0078'),
 ('1902', '-0089'),
 ('1902', '-0122'),
 ('1902', '-0139'),
 ('1902', '-0139'),
 ('1902', '-0144'),
 ('1902', '-0139'),
 ('1902', '-0078'),
 ('1902', '-0017'),
 ('1902', '-0011'),
 ('1902', '-0044'),
 ('1902', '-0044'),
 ('1902', '-0061'),


In [19]:
max_temperature_per_year = temperature_per_year.reduceByKey(lambda x,y: int(x) if int(x)>int(y) else int(y))

In [20]:
max_temperature_per_year.collect()

[('1902', 244)]

In [21]:
years = max_temperature_per_year.map(lambda x: x[1])
years.distinct().collect()

[244]

In [32]:
#Save the RDD back into HDFS
max_temperature_per_year.saveAsTextFile("out/output")

Py4JJavaError: An error occurred while calling o201.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/data/lectures/ttmda/spark/out/output already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:108)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:306)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.

In [33]:
weatherData_ = sc.textFile("out/output")

In [34]:
weatherData_.collect()

["('1902', 244)"]